In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
from konlpy.tag import Mecab
import re
from konlpy.tag import Twitter
from sklearn.model_selection import train_test_split
twitter = Twitter()
from collections import namedtuple
from keras.preprocessing import sequence

/anaconda3/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
Using TensorFlow backend.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

from torchtext import data
from torchtext import datasets

from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import ProgressBar
#import Early Stopping
from pytorchtools import EarlyStopping

In [3]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load('slang_word2vec.model')

In [4]:
dataset = pd.read_csv("/Users/SAEROM/slang_dict_twitter(1).csv", encoding='cp949')
#첫 5행을 불러옴
dataset.head() 

,word,frequency,dict,class
0,새끼,64,1,1
1,남자,37,1,0
2,진짜,36,1,0
3,여자,32,1,0
4,사람,27,1,0


In [5]:
train, test = train_test_split(dataset, test_size=0.2, shuffle = True)
train, valid = train_test_split(train, test_size=0.2, shuffle = True)
print(len(train), len(valid), len(test))

640 160 200


In [6]:
class Data(torch.utils.data.Dataset):
    def __init__(self, data):
        #self.data=[]
        #self.label=[]
        
        #word,freq,dict,tag라는 라벨을 갖고 있는 namedtuple 클래스를 생성.
        TaggedWord = namedtuple('TaggedWord', 'word freq dict tag')
        tagged_words = [TaggedWord(w, f, d, c) for w,f,d,c 
                              in data[["word", "frequency", "dict", "class"]].values]
        #print(tagged_words)
        data_x = []
        data_y = []
        
        vector_=[]
        for t in tagged_words:
            if t.word in w2v_model: 
                vector_ = []
                vector = w2v_model.wv[t.word]
                #vector_.append(vector)
                #vector_.append(t.freq)
                #vector_.append(t.dict)
                for v in vector:
                    vector_.append(v)
                vector_.append(t.freq)
                vector_.append(t.dict)
            
        
            data_x.append(vector_)
            data_y.append(t.tag)
        #print(data_x)
        #print(data_y)

        #print(type(np.array(data_x)))
        #np.array(data_y)
        
        self.data = np.array(data_x)
        self.label = np.array(data_y)
        
        print("<데이터의 shape>")
        print(type(self.data))
        print(np.array(self.data).shape)
        print("<데이터 내용>") 
        print(torch.tensor(self.data[0], dtype=torch.float))
        print("<레이블 표시>") 
        print(self.label[0])
        print("<총 레이블 길이>") 
        print(len(self.label)) 
        
    def __getitem__(self, index):
        return torch.tensor(self.data[index], dtype=torch.float), self.label[index]
    
    def __len__(self):
        return len(self.data)

In [7]:
class DNN (nn.Module):
    def __init__(self):
        super (DNN, self).__init__ ()
        
        self.fc1 = nn.Sequential(
            nn.Linear (102,64),
            nn.ELU())
        self.fc2 = nn.Sequential(
            nn.Linear (64, 32),
            nn.ELU())
        self.fc3 = nn.Sequential(
            nn.Linear (32, 2),
            nn.ELU())
        self.dropout = nn.Dropout(0.5)
        
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        x = x.view (-1, 102)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        
        x = self.softmax(x)
        
        return x
model = DNN()
print(model)

DNN(
  (fc1): Sequential(
    (0): Linear(in_features=102, out_features=64, bias=True)
    (1): ELU(alpha=1.0)
  )
  (fc2): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ELU(alpha=1.0)
  )
  (fc3): Sequential(
    (0): Linear(in_features=32, out_features=2, bias=True)
    (1): ELU(alpha=1.0)
  )
  (dropout): Dropout(p=0.5)
  (softmax): Softmax()
)


In [8]:
# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.Adam(model.parameters())

In [9]:
from pytorchtools import EarlyStopping

In [10]:
def train_model (model, batch_size, patience, epochs): 
    torch.manual_seed(42)
    
    ##################################################
    #train and validate model#
    ##################################################
    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 
  

    ###################
    # train the model #
    ###################  
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)
        
    for epoch in range(1, epochs+1):
        for batch, (data, label) in enumerate (loader_train, 1):
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, label)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())
            
    ###################
    # validate the model #
    ###################  
        model.eval()
        for data, label in loader_valid:
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, label)
            # record validation loss
            valid_losses.append(loss.item())
    
        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
            
    return  model, avg_train_losses, avg_valid_losses

In [11]:
data_train = Data (data = train)
loader_train = torch.utils.data.DataLoader(data_train, batch_size = 20, shuffle = False)
data_test = Data (data = test)
loader_test = torch.utils.data.DataLoader(data_test, batch_size = 20, shuffle = False)
data_valid = Data (data = valid)
loader_valid = torch.utils.data.DataLoader(data_valid, batch_size = 20, shuffle = False)

<데이터의 shape>
<class 'numpy.ndarray'>
(640, 102)
<데이터 내용>
tensor([ 0.3819,  0.2132, -0.3604,  0.2379, -0.1995,  0.3333,  0.0216, -0.7131,
        -0.3337,  0.8733, -0.0238, -0.5600,  0.0390,  0.3155, -0.3521, -0.4481,
         0.3581, -0.0848, -0.2443, -0.5533,  0.7288,  0.1514, -0.3361, -0.1244,
         0.4870, -0.1097,  0.2365, -0.4217, -0.4788, -0.1319,  0.6424,  0.3803,
         0.2332,  0.4452, -0.2730,  0.2229,  0.1602, -0.2200,  1.0953,  0.9725,
        -0.2804, -0.4278, -1.0465, -0.1070,  0.3589, -0.5911, -0.1063,  0.1173,
        -0.1100,  0.0822, -0.1169, -0.9469,  0.1719, -0.3450, -0.0824, -1.0463,
        -0.4829,  0.6174, -0.2881,  0.6837,  0.2494, -0.0600, -0.0855, -0.7763,
        -0.2366,  0.0754,  0.2075,  0.0589,  0.5708,  0.3120, -0.5714, -0.0172,
         0.5878,  0.1189,  0.4924,  0.6758,  0.0679, -0.2885,  0.6142, -0.3036,
         0.1036, -0.9548,  0.3040, -0.2315, -0.7264, -0.6967,  0.4627,  0.1315,
         0.2449,  0.2645,  0.5762,  0.4029,  0.0325,  0.2728,  

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()


In [12]:
# early stopping patience; how long to wait after last time validation loss improved.
#patience = 20
model, train_loss, valid_loss = train_model(model, 
                                            batch_size=20,epochs=100 , patience = 20)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[  1/100] train_loss: 0.59670 valid_loss: 0.52766
Validation loss decreased (inf --> 0.527665).  Saving model ...
[  2/100] train_loss: 0.52789 valid_loss: 0.51109
Validation loss decreased (0.527665 --> 0.511095).  Saving model ...
[  3/100] train_loss: 0.52310 valid_loss: 0.50925
Validation loss decreased (0.511095 --> 0.509252).  Saving model ...
[  4/100] train_loss: 0.52205 valid_loss: 0.50858
Validation loss decreased (0.509252 --> 0.508583).  Saving model ...
[  5/100] train_loss: 0.52087 valid_loss: 0.50814
Validation loss decreased (0.508583 --> 0.508139).  Saving model ...
[  6/100] train_loss: 0.51599 valid_loss: 0.50808
Validation loss decreased (0.508139 --> 0.508081).  Saving model ...
[  7/100] train_loss: 0.50214 valid_loss: 0.50187
Validation loss decreased (0.508081 --> 0.501873).  Saving model ...
[  8/100] train_loss: 0.49221 valid_loss: 0.49819
Validation loss decreased (0.501873 --> 0.498186).  Saving model ...
[  9/100] train_loss: 0.48274 valid_loss: 0.49644
Val

In [13]:
def test_model(model, batch_size):
    torch.manual_seed(42)
    
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    model.eval() # prep model for evaluation

    for data, target in loader_test:
        if len(target.data) != batch_size:
            break
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        # calculate test accuracy for each object class
        print(target)
        print(data)
        print(pred)
        for i in range(batch_size):
            label = target.data[i]
            #print('#############')
            #print(, end=' ')
            #print(target.data.view_as)
            #print(loader_test.dataset)
            #print("\t", label, "\t" , target.data.view_as(pred))
            class_correct[label] += correct[i].item()
            class_total[label] += 1

        # calculate and print avg test loss
        test_loss = test_loss/len(test)
        print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                str(i), 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        #else:
        #    print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    
    
    
    #return target.data, label, correct
        

In [14]:
test_model(model, batch_size=20)

tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
tensor([[-0.3199,  0.0274, -0.3871,  ...,  0.2444,  1.0000,  0.0000],
        [ 0.7787, -0.1147,  0.5204,  ..., -0.1212, 21.0000,  1.0000],
        [-0.1319, -0.1194, -0.3521,  ...,  0.2763,  1.0000,  1.0000],
        ...,
        [ 0.3970,  1.0212, -1.0551,  ...,  0.3575,  9.0000,  1.0000],
        [-0.6048,  0.4381, -0.4709,  ...,  0.3205,  1.0000,  1.0000],
        [ 0.1822,  0.0225, -0.5468,  ...,  0.6620,  6.0000,  0.0000]])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
Test Loss: 0.036337

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
tensor([[-4.2895e-01, -4.8796e-01, -1.9639e-01,  ..., -8.2448e-02,
          2.6000e+01,  1.0000e+00],
        [ 2.1340e-01,  1.5919e-01, -9.6772e-01,  ...,  4.0688e-02,
          7.0000e+00,  1.0000e+00],
        [-4.6134e-01,  6.6106e-01, -5.1957e-01,  ...,  2.0994e-01,
          1.0000e+00,  1.0000e+00],
        ...,
        [-9.0150

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [15]:
TaggedWord = namedtuple('TaggedWord', 'word freq dict tag')
tagged_words = [TaggedWord(w, f, d, c) for w,f,d,c 
                in test[["word", "frequency", "dict", "class"]].values]
for test in tagged_words:
    if test.word in w2v_model: 
        print( "단어 : " ,test.word, "\t실제 레이블 : " ,test.tag ,"\t예측 레이블 : ")

단어 :  웨딩피치 	실제 레이블 :  1 	예측 레이블 : 
단어 :  년 	실제 레이블 :  1 	예측 레이블 : 
단어 :  진보 	실제 레이블 :  0 	예측 레이블 : 
단어 :  간호사 	실제 레이블 :  0 	예측 레이블 : 
단어 :  나이 	실제 레이블 :  0 	예측 레이블 : 
단어 :  해결 	실제 레이블 :  0 	예측 레이블 : 
단어 :  길 	실제 레이블 :  0 	예측 레이블 : 
단어 :  아줌마 	실제 레이블 :  0 	예측 레이블 : 
단어 :  기술 	실제 레이블 :  0 	예측 레이블 : 
단어 :  부모님 	실제 레이블 :  0 	예측 레이블 : 
단어 :  막상 	실제 레이블 :  0 	예측 레이블 : 
단어 :  물건 	실제 레이블 :  0 	예측 레이블 : 
단어 :  대학가 	실제 레이블 :  0 	예측 레이블 : 
단어 :  위로 	실제 레이블 :  0 	예측 레이블 : 
단어 :  문화 	실제 레이블 :  0 	예측 레이블 : 
단어 :  헬무새 	실제 레이블 :  1 	예측 레이블 : 
단어 :  한숨 	실제 레이블 :  0 	예측 레이블 : 
단어 :  돈 	실제 레이블 :  0 	예측 레이블 : 
단어 :  경제 	실제 레이블 :  0 	예측 레이블 : 
단어 :  좌좀 	실제 레이블 :  1 	예측 레이블 : 
단어 :  거 	실제 레이블 :  0 	예측 레이블 : 
단어 :  이해 	실제 레이블 :  0 	예측 레이블 : 
단어 :  혹시 	실제 레이블 :  0 	예측 레이블 : 
단어 :  부터 	실제 레이블 :  0 	예측 레이블 : 
단어 :  눈치 	실제 레이블 :  0 	예측 레이블 : 
단어 :  년놈 	실제 레이블 :  1 	예측 레이블 : 
단어 :  때 	실제 레이블 :  0 	예측 레이블 : 
단어 :  심 	실제 레이블 :  0 	예측 레이블 : 
단어 :  저런 	실제 레이블 :  0 	예측 레이블 : 
단어 :  도중 	실제 레이블 :  0 	예측 레이블 : 
단어 :  떡 	

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """


	실제 레이블 :  0 	예측 레이블 : 
단어 :  더 	실제 레이블 :  0 	예측 레이블 : 
단어 :  강매 	실제 레이블 :  0 	예측 레이블 : 
단어 :  이불 	실제 레이블 :  0 	예측 레이블 : 
단어 :  그때 	실제 레이블 :  0 	예측 레이블 : 
단어 :  마냥 	실제 레이블 :  0 	예측 레이블 : 
단어 :  인생 	실제 레이블 :  0 	예측 레이블 : 
단어 :  위축 	실제 레이블 :  0 	예측 레이블 : 
단어 :  피해 	실제 레이블 :  0 	예측 레이블 : 
단어 :  대처 	실제 레이블 :  0 	예측 레이블 : 
단어 :  방식 	실제 레이블 :  0 	예측 레이블 : 
단어 :  삼수 	실제 레이블 :  1 	예측 레이블 : 
단어 :  고양이 	실제 레이블 :  0 	예측 레이블 : 
단어 :  몸 	실제 레이블 :  0 	예측 레이블 : 
단어 :  사람 	실제 레이블 :  0 	예측 레이블 : 
단어 :  성매매 	실제 레이블 :  1 	예측 레이블 : 
단어 :  무엇 	실제 레이블 :  0 	예측 레이블 : 
단어 :  낫다 	실제 레이블 :  0 	예측 레이블 : 
단어 :  팩트 	실제 레이블 :  1 	예측 레이블 : 
단어 :  자고 	실제 레이블 :  0 	예측 레이블 : 
단어 :  시비 	실제 레이블 :  0 	예측 레이블 : 
단어 :  일단 	실제 레이블 :  0 	예측 레이블 : 
단어 :  인스타 	실제 레이블 :  1 	예측 레이블 : 
단어 :  중독 	실제 레이블 :  0 	예측 레이블 : 
단어 :  여우 	실제 레이블 :  0 	예측 레이블 : 
단어 :  도용 	실제 레이블 :  0 	예측 레이블 : 
단어 :  압살 	실제 레이블 :  0 	예측 레이블 : 
단어 :  살해 	실제 레이블 :  0 	예측 레이블 : 
단어 :  대신 	실제 레이블 :  0 	예측 레이블 : 
단어 :  들 	실제 레이블 :  0 	예측 레이블 : 
단어 :  녹음 	실제 레이블 : 